# 2.1.1 SRA 

In [8]:
import os
os.chdir("SRA")

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

## 1 Retrieve SRA metadata

Code to connect Assembly Ids (Genbank) with SRA metadata

## 2 Upload SRA metadata

In [3]:
sra = pd.read_csv("reads_dataset.txt", sep="\t")

In [4]:
code = pd.read_csv("assembly_srr.txt", sep="\t", names=["assembly_accession", "run_accession"])
code.head(2)

,assembly_accession,run_accession
0,GCA_000023285.1,SRR013476
1,GCA_000023285.1,SRR013477


In [5]:
code = code.drop_duplicates(subset=['assembly_accession']) 

In [6]:
code_sra = pd.merge(code, sra, on="run_accession")

In [7]:
code_sra.tail(2)

,assembly_accession,run_accession,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,experiment_ID,experiment_alias,experiment_accession,experiment_title,study_name,sample_name,design_description,library_name,library_strategy,library_source,library_selection,library_layout,library_construction_protocol,read_spec,platform,instrument_model,platform_parameters,experiment_attribute,sample_ID,sample_alias,sample_accession,taxon_id,common_name,description,sample_url_link,sample_attribute,study_ID,study_alias,study_accession,study_title,study_type,study_abstract,center_project_name,study_description,study_url_link,study_attribute,submission_ID,submission_accession,submission_comment,submission_center,submission_lab,submission_date,sradb_updated
534,GCA_902804475.1,ERR3888428,0,9069339,ena-RUN-INSTITUT PASTEUR-06-02-2020-13:43:47:1...,NaN,2020-02-25,0.0,0.0,NaN,NaN,ENA-FIRST-PUBLIC: 2020-02-21 || ENA-LAST-UPDAT...,8233124,ena-EXPERIMENT-INSTITUT PASTEUR-06-02-2020-13:...,ERX3898656,NextSeq 500 paired end sequencing,NaN,NaN,NaN,Nextera XT,WGS,GENOMIC,RANDOM,PAIRED - NOMINAL_LENGTH: 317;,NaN,NaN,ILLUMINA,NextSeq 500,INSTRUMENT_MODEL: NextSeq 500,NaN,7432552.0,NaN,ERS4290983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ERP119591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1567020,ERA2376428,NaN,NaN,European Nucleotide Archive,NaN,2020-09-03 11:01:35
535,GCA_902804485.1,ERR3888429,0,9069340,ena-RUN-INSTITUT PASTEUR-06-02-2020-13:43:47:1...,NaN,2020-02-25,0.0,0.0,NaN,NaN,ENA-FIRST-PUBLIC: 2020-02-21 || ENA-LAST-UPDAT...,8233125,ena-EXPERIMENT-INSTITUT PASTEUR-06-02-2020-13:...,ERX3898657,NextSeq 500 paired end sequencing,NaN,NaN,NaN,Nextera XT,WGS,GENOMIC,RANDOM,PAIRED - NOMINAL_LENGTH: 327;,NaN,NaN,ILLUMINA,NextSeq 500,INSTRUMENT_MODEL: NextSeq 500,NaN,7432551.0,NaN,ERS4290982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ERP119591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1567020,ERA2376428,NaN,NaN,European Nucleotide Archive,NaN,2020-09-03 11:01:35


## 3 Solve sample attribute column

In [8]:
att = pd.DataFrame(code_sra["sample_attribute"])
att.head(2)

,sample_attribute
0,strain: DSM 7271 || type-material: type strain...
1,strain: ATCC 35910 || misc_param: HMP supersit...


In [9]:
att.to_csv("att.csv")

In [10]:
d = {}
l = []

f = open("att.csv", "r")
lines = f.readlines()

for line in lines:
    splited = line.split(r" || ")
    for x in splited:
        l.append(x)

for item in l:
    a = item.split(": ")
    key = a[0]
    key = key.replace('"', '')
    key = key.replace(' ', '')
    value = a[1:]
    if key in d:
        # append the new number to the existing array at this slot
        d[key].append(value)
    else:
        # create a new array in this slot
        d[key] = [value]

import pandas as pd

df = pd.DataFrame.from_dict(d, orient='index').T
df.to_csv("from_script.csv", index=False)

f.close()

In [11]:
att2 = pd.read_csv("from_script.csv")

In [12]:
att2.replace(regex=True,inplace=True,to_replace=r"\['",value=r'')
att2.replace(regex=True,inplace=True,to_replace=r"\']",value=r'')
att2.head(2)

,",sample_attribute\n","0,strain",type-material,"1,strain",misc_param,project_type,estimated_size,culture-collection,feature,investigation_type,hosttaxid,nucleicacidextraction,finishingstrategy(depthofcoverage),material,sop,num_replicons,ref_biomaterial,geo_loc_name,biome,latlon,env_package,collectiondate,specifichost,sequencingmethod,assembly,isol_growth_condt,source_mat_id,project_name,isolation-source,BioSampleModel,"2,finishingstrategy(depthofcoverage)",lat_lon,investigationtype,strain,environmentalpackage,"3,finishingstrategy(depthofcoverage)","4,biotic_relationship",IsolationSite,TemperatureOptimum,BodySampleSite,Motility,GOLDStampID,Diseases,collection_date,environment,IsolationComments,TemperatureRange,TypeStrain,rel_to_oxygen,alt_elev,FundingProgram,Sporulation,trophic_level,country,depth,CellShape,HostHealth,HostName,"5,finishingstrategy(depthofcoverage)","6,biotic_relationship",pH,GramStaining,GeneCallingMethod,"7,\n","8,\n","9,finishingstrategy(depthofcoverage)","10,\n","11,\n","12,finishingstrategy(depthofcoverage)","13,env_package",biotic_relationship,"14,biotic_relationship","15,env_package","16,biotic_relationship",Isolationsource,Host,geo-loc-name,BodySampleSubSite,"17,finishingstrategy(depthofcoverage)","18,finishingstrategy(depthofcoverage)","19,finishingstrategy(depthofcoverage)","20,strain","21,strain","22,finishingstrategy(depthofcoverage)","23,strain","24,strain",host,"25,env_package",isolationsource,"26,strain",note,"27,strain","28,strain","29,strain",Phenotypes,"30,strain","31,strain","32,strain","33,strain","34,strain",...,"467,strain","468,strain","469,strain","470,strain","471,strain","472,geo_loc_name","473,strain","474,strain","475,geo_loc_name","476,synonym",geographiclocation,latitudeandlongitude,ENA-FIRST-PUBLIC,ENA-LAST-UPDATE,"477,synonym","478,synonym","479,strain",sample_description,specific_host,anonymized_name,host_status,supplier_name,"480,strain","481,strain","482,geo_loc_name","483,strain","484,synonym","485,strain","486,strain","487,strain","488,strain","489,\n","490,strain",ArrayExpress-StrainOrLine,ArrayExpress-Species,"491,\n","492,strain","493,strain","494,\n","495,strain",ArrayExpress-Phenotype,"496,Strain",SampleDescription,"497,\n","498,\n","499,Strain","500,\n","501,\n","502,\n","503,Strain","504,\n","505,\n","506,strain","507,strain","508,\n","509,strain","510,\n","511,strain","512,strain","513,\n","514,\n","515,\n","516,\n","517,\n","518,strain","519,strain","520,strain","521,strain","522,SUBJECT_ID",ArrayExpress-STRAIN_OR_LINE,ArrayExpress-SPECIES,"523,\n","524,\n","525,\n","526,\n","527,Alias",ENAchecklist,INSDCcenteralias,INSDCcentername,INSDCfirstpublic,INSDClastupdate,INSDCstatus,SRAaccession,SampleName,Title,environment(biome),environment(feature),environment(material),geographiclocation(latitude),geographiclocation(longitude),plant-associatedenvironmentalpackage,projectname,"528,Alias","529,Alias","530,collected_by","531,\n","532,\n","533,\n","534,\n","535,\n"
0,[],DSM 7271,type strain of Capnocytophaga ochracea\n,ATCC 35910,"HMP supersite', 'urogenital_tract",Reference Genome,5562000,ATCC:35910,Homo sapiens-associated habitat [ENVO:00009003],missing,9606,not determined,"Level 6', 'Finished33.84x;7",biological product [ENVO:02000043],http://hmpdacc.org/doc/CommonGeneAnnotation_SO...,not determined,not determined,not determined,terrestrial biome [ENVO:00000446],not determined,missing,not determined,Homo sapiens,"454-GS-FLX, Illumina",Newbler v. 2.0.0-Eclipse,not determined,"ATCC 35910, CL4/79, IFO 15054, JCM 2410, NCTC ...","Chryseobacterium gleum F93, ATCC 35910",missing,"MIGS.ba""\n","Level 2', 'High-Quality Draft19x;65",not determined,missing,Capno,missing,"Level 2', 'High-Quality Draft37x;37",symbiont,Duck blood,37C,Blood,Nonmotile,Gi04613,Septicemia,1954,Missing,isolated by Bruner and Fabricant,Mesophile,Yes,microaerophilic,Missing,DOE-GEBA 2007,Nonsporulating,Missing,USA,Missing,Rod-shaped,Septicemic disease in waterfowls. Septicemia a...,"Anas platyrhynchos (duc

#### 3.1 Get counts:

In [13]:
d ={}
for col in att2.columns:
    tmp_series = att2[col].value_counts()
    tmp_series.name = col
    d[col]=tmp_series

In [14]:
df = pd.concat(d.values(), keys=d.keys())
print(type(df))

<class 'pandas.core.series.Series'>


In [15]:
df.to_csv("counts_final.csv", header=["count"])

#### 3.2 Sample attribute division per columns

In [16]:
sample_att = code_sra.copy()
sample_att2 = sample_att["sample_attribute"]

In [17]:
sample_att2 = sample_att[["assembly_accession","sample_attribute"]]

In [18]:
sample_att2["sample_attribute"] = sample_att2.sample_attribute.str.replace(r" \|\| ", ",")
sample_att2.head(2)

/home/gomes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,assembly_accession,sample_attribute
0,GCA_000023285.1,"strain: DSM 7271,type-material: type strain of..."
1,GCA_000143785.1,"strain: ATCC 35910,misc_param: HMP supersite: ..."


In [19]:
assembly_accession = sample_att2["assembly_accession"].to_list()

In [20]:
splitted = sample_att2.sample_attribute.str.split(",", expand=True)
splitted.drop(splitted.tail(2).index,inplace=True)
splitted = splitted.rename(index=sample_att["assembly_accession"])

In [21]:
col_list2 = []
for i, cell in splitted.iterrows():
    cell2 = cell[0]
    cell2 = str(cell2)
    key = cell2.split(": ")[0]
    value = cell2.split(": ")[1:]
    if key not in col_list2:
        col_list2.append(key)

In [22]:
for i, cell in splitted.iteritems():
    for x in cell:
        if x is None:
            continue
        else:
            x = str(x)
            if ":" in x:
                key = x.split(": ")[0]
                #print(key)
                #value = x.split(": ")[1:]
                if key not in col_list2:
                     col_list2.append(key)

In [23]:
df = pd.DataFrame(columns=col_list2, index=sample_att["assembly_accession"])
df.head(2)

,strain,finishing strategy (depth of coverage),biotic_relationship,nan,env_package,geo_loc_name,host,collected_by,host_disease,isolation_source,sample_name,isolation-source,sample_type,FDA CVM ID,culture_collection,assembly_method,collection_date,env_feature,synonym,Strain,SUBJECT_ID,Alias,type-material,misc_param,collection date,Isolation Site,Isolation source,country,note,GOLD Stamp ID,investigation_type,culture-collection,lat_lon,alt_elev,locus_tag_prefix,nat-host,assembly_method_version,env_material,model,sample_description,Sample Description,ArrayExpress-STRAIN_OR_LINE,ENA checklist,project_type,estimated_size,Temperature Optimum,isol_growth_condt,Motility,feature,project_name,assembly,num_replicons,isolate,depth,env_biome,altitude,ref_biomaterial,package,geographic location,ArrayExpress-StrainOrLine,ArrayExpress-Phenotype,ArrayExpress-SPECIES,INSDC center alias,geo-loc-name,environment,host_sex,elev,PublicAccession,completeness_estimated,geographic location (country and/or sea),latitude and longitude,specific_host,ArrayExpress-Species,ENA-FIRST-PUBLIC,INSDC center name,specific host,Body Sample Site,pH,biome,biomaterial_provider,isolate_name_alias,contamination_estimated,host scientific name,anonymized_name,ENA-LAST-UPDATE,INSDC first public,Cell Shape,BioSampleModel,environmental_sample,material,identified_by,host health state,INSDC last update,lat lon,Diseases,strain_name_alias,genome_coverage,host_status,INSDC status,host taxid,isolation source,Gram Staining,Temperature Range,Sporulation,sequencing_meth,Historical Monobactam Produced,identification method,supplier_name,SRA accession,nucleic acid extraction,host_disease_outcome,Unique Monobactam Clusters Identified,ProjectAccession,Sample Name,Type Strain,Phenotypes,Gene Calling Method,temp,Title,investigation type,Host,rel_to_oxygen,host_age,Species,mapping_method,serovar,sop,Isolation Comments,finishing_strategy,Genus,mapping_method_version,environment (biome),attribute_package,metagenome_source,environment (feature),trophic_level,metagenomic,environment (material),Funding Program,quality_assessment_method,host_health_state,quality_assessment_method_version,geographic location (latitude),source_mat_id,host_tissue_sampled,host_description,geographic location (longitude),Host Name,host_disease_stage,subsrc_note,Host Health,value,Laboratory Host,plant-associated environmental package,Extraction Method,project name,sequencing method,Body Sample SubSite,passage_history,Extraction Date,sample type,environmental package,Passage Date
assembly_accession,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GCA_000023285.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GCA_000143785.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [24]:
for i, cell in splitted.iterrows():
    for x in cell:
        cell = x
        if type(cell) != None:
            cell = str(cell)
            cell2 = cell[0]
            #print(cell)
            key = cell.split(": ")[0]#
            if key in col_list2:
                value = cell.split(": ")[1:]
                df.loc[i,key] = str(value)

In [25]:
d = {r'\[':'', r'\]':r''}
df.replace(d, regex=True, inplace=True)
df.drop(labels=["nan"],axis=1, inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536 entries, GCA_000023285.1 to GCA_902804485.1
Columns: 162 entries, strain to Passage Date
dtypes: object(162)
memory usage: 702.6+ KB


In [27]:
df.replace("'", "", regex=True, inplace=True)
df.replace(to_replace=["missing", "Missing", "Unclassified", "NA", "not determined", "Not collected", 
                                          "unspecified", "not applicable", "Unknown", "Not applicable", "'Missing'",
                                          "Not Applicable", "not collected"], value= None, regex=True, inplace=True)
df.head(2)

,strain,finishing strategy (depth of coverage),biotic_relationship,env_package,geo_loc_name,host,collected_by,host_disease,isolation_source,sample_name,isolation-source,sample_type,FDA CVM ID,culture_collection,assembly_method,collection_date,env_feature,synonym,Strain,SUBJECT_ID,Alias,type-material,misc_param,collection date,Isolation Site,Isolation source,country,note,GOLD Stamp ID,investigation_type,culture-collection,lat_lon,alt_elev,locus_tag_prefix,nat-host,assembly_method_version,env_material,model,sample_description,Sample Description,ArrayExpress-STRAIN_OR_LINE,ENA checklist,project_type,estimated_size,Temperature Optimum,isol_growth_condt,Motility,feature,project_name,assembly,num_replicons,isolate,depth,env_biome,altitude,ref_biomaterial,package,geographic location,ArrayExpress-StrainOrLine,ArrayExpress-Phenotype,ArrayExpress-SPECIES,INSDC center alias,geo-loc-name,environment,host_sex,elev,PublicAccession,completeness_estimated,geographic location (country and/or sea),latitude and longitude,specific_host,ArrayExpress-Species,ENA-FIRST-PUBLIC,INSDC center name,specific host,Body Sample Site,pH,biome,biomaterial_provider,isolate_name_alias,contamination_estimated,host scientific name,anonymized_name,ENA-LAST-UPDATE,INSDC first public,Cell Shape,BioSampleModel,environmental_sample,material,identified_by,host health state,INSDC last update,lat lon,Diseases,strain_name_alias,genome_coverage,host_status,INSDC status,host taxid,isolation source,Gram Staining,Temperature Range,Sporulation,sequencing_meth,Historical Monobactam Produced,identification method,supplier_name,SRA accession,nucleic acid extraction,host_disease_outcome,Unique Monobactam Clusters Identified,ProjectAccession,Sample Name,Type Strain,Phenotypes,Gene Calling Method,temp,Title,investigation type,Host,rel_to_oxygen,host_age,Species,mapping_method,serovar,sop,Isolation Comments,finishing_strategy,Genus,mapping_method_version,environment (biome),attribute_package,metagenome_source,environment (feature),trophic_level,metagenomic,environment (material),Funding Program,quality_assessment_method,host_health_state,quality_assessment_method_version,geographic location (latitude),source_mat_id,host_tissue_sampled,host_description,geographic location (longitude),Host Name,host_disease_stage,subsrc_note,Host Health,value,Laboratory Host,plant-associated environmental package,Extraction Method,project name,sequencing method,Body Sample SubSite,passage_history,Extraction Date,sample type,environmental package,Passage Date
assembly_accession,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GCA_000023285.1,DSM 7271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Capnocytophaga ochracea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GCA_000143785.1,ATCC 35910,"Level 6, Finished33.84x;7",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Chryseobacterium gleum,"HMP body site, Vaginal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATCC:35910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reference Genome,5562000,NaN,NaN,NaN,Homo sapiens-associated habitat ENVO:00009003,Chryseobacterium gleum F93,Newbler v. 2.0.0-Eclipse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homo sapiens,NaN,NaN,terrestrial biome ENVO:00000446,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [28]:
null_cols = df.isnull().all()
df = df[df.columns[~null_cols]]

In [29]:
#To remove white space at the beginning of string:
df.columns = df.columns.str.lstrip()

#To remove white space at the end of string:
df.columns = df.columns.str.rstrip()

In [30]:
cols = df.columns.tolist()

In [31]:
colstoremove =[]
for x in cols:
    a = str(x)
    if (604 - len(df[df[a].isnull()])) <= 2:
        colstoremove.append(x)

In [32]:
print(colstoremove)

[]


In [33]:
df.drop(colstoremove, axis = 1, inplace=True) 

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536 entries, GCA_000023285.1 to GCA_902804485.1
Columns: 148 entries, strain to Body Sample SubSite
dtypes: object(148)
memory usage: 623.9+ KB


## 4 Unify SRA metadata

In [35]:
code_sra.head(2)

,assembly_accession,run_accession,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,experiment_ID,experiment_alias,experiment_accession,experiment_title,study_name,sample_name,design_description,library_name,library_strategy,library_source,library_selection,library_layout,library_construction_protocol,read_spec,platform,instrument_model,platform_parameters,experiment_attribute,sample_ID,sample_alias,sample_accession,taxon_id,common_name,description,sample_url_link,sample_attribute,study_ID,study_alias,study_accession,study_title,study_type,study_abstract,center_project_name,study_description,study_url_link,study_attribute,submission_ID,submission_accession,submission_comment,submission_center,submission_lab,submission_date,sradb_updated
0,GCA_000023285.1,SRR013476,0,1093152,FWCBa,NaN,2019-11-28,9047.0,2866262.0,JGI,FWCBa,flow_count: 110 || flow_sequence: TACG || TCAG...,1051231,FWCBa,SRX002589,NaN,FWCB,FWCB,none provided,FWCB,WGS,GENOMIC,RANDOM,SINGLE -,NaN,READ_INDEX: 0; READ_CLASS: Technical Read; REA...,LS454,454 GS FLX,INSTRUMENT_MODEL: 454 GS FLX,NaN,780997.0,FWCB,SRS002078,521097.0,NaN,Capnocytophaga ochracea DSM 7271,NaN,strain: DSM 7271 || type-material: type strain...,16818.0,FWCB,SRP000584,Causes periodontitis,Whole Genome Sequencing,<P><B><I>Capnocytophaga ochracea</I> DSM 7271<...,Capnocytophaga ochracea DSM 7271,NaN,NaN,parent_bioproject: PRJNA30815,310018,SRA008146,FTP,JGI,PGF,2009-02-19,2020-09-03 11:01:35
1,GCA_000143785.1,SRR004314,0,1083352,2007-08-02.1147-FLX01070128,NaN,2019-11-28,158268.0,44181818.0,NaN,4WG_HMP0003.CGLE.01_000sA,run_center: BCM,1048730,4WG_HMP0003.CGLE.01_000sA,SRX001228,454 sequencing of Chryseobacterium gleum ATCC ...,NaN,NaN,FLX fragment,4WG_HMP0003.CGLE.01_000sA,WGS,GENOMIC,RANDOM,SINGLE -,NaN,READ_INDEX: 0; READ_CLASS: Technical Read; REA...,LS454,454 GS FLX,INSTRUMENT_MODEL: 454 GS FLX,NaN,779023.0,BCM-HMP0003,SRS000860,525257.0,NaN,Chryseobacterium gleum ATCC 35910,BCM HMP Project: http://www.hgsc.bcm.edu/micro...,strain: ATCC 35910 || misc_param: HMP supersit...,16258.0,WG_HMP0003.CGLE.01_000,SRP000258,Reference genome for the Human Microbiome Project,Whole Genome Sequencing,<P> <I><B>Chryseobacterium gleum</I> ATCC 3591...,Chryseobacterium gleum ATCC 35910,NaN,NaN,parent_bioproject: PRJNA28331,304555,SRA002043,NaN,BCM,NaN,2008-09-11,2020-09-03 11:01:35


In [50]:
df = df.reset_index()
df.head(2)

,assembly_accession,strain,finishing strategy (depth of coverage),biotic_relationship,geo_loc_name,host,collected_by,host_disease,isolation_source,sample_name,isolation-source,sample_type,FDA CVM ID,culture_collection,assembly_method,collection_date,env_feature,synonym,Strain,SUBJECT_ID,Alias,type-material,misc_param,collection date,Isolation Site,Isolation source,country,note,GOLD Stamp ID,investigation_type,culture-collection,lat_lon,alt_elev,locus_tag_prefix,nat-host,assembly_method_version,env_material,model,sample_description,Sample Description,ArrayExpress-STRAIN_OR_LINE,ENA checklist,project_type,estimated_size,Temperature Optimum,isol_growth_condt,Motility,feature,project_name,assembly,num_replicons,isolate,depth,env_biome,altitude,ref_biomaterial,package,geographic location,ArrayExpress-StrainOrLine,ArrayExpress-Phenotype,ArrayExpress-SPECIES,INSDC center alias,geo-loc-name,environment,host_sex,elev,PublicAccession,completeness_estimated,geographic location (country and/or sea),latitude and longitude,specific_host,ArrayExpress-Species,ENA-FIRST-PUBLIC,INSDC center name,specific host,Body Sample Site,pH,biome,biomaterial_provider,isolate_name_alias,contamination_estimated,host scientific name,anonymized_name,ENA-LAST-UPDATE,INSDC first public,Cell Shape,BioSampleModel,environmental_sample,material,identified_by,host health state,INSDC last update,Diseases,strain_name_alias,genome_coverage,INSDC status,host taxid,isolation source,Gram Staining,Temperature Range,Sporulation,sequencing_meth,Historical Monobactam Produced,identification method,supplier_name,SRA accession,host_disease_outcome,Unique Monobactam Clusters Identified,ProjectAccession,Sample Name,Type Strain,Phenotypes,Gene Calling Method,temp,Title,investigation type,Host,rel_to_oxygen,host_age,Species,mapping_method,serovar,sop,Isolation Comments,finishing_strategy,Genus,mapping_method_version,environment (biome),attribute_package,metagenome_source,environment (feature),trophic_level,metagenomic,environment (material),Funding Program,quality_assessment_method,quality_assessment_method_version,geographic location (latitude),source_mat_id,host_tissue_sampled,geographic location (longitude),Host Name,subsrc_note,Host Health,value,plant-associated environmental package,project name,sequencing method,Body Sample SubSite
0,GCA_000023285.1,DSM 7271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Capnocytophaga ochracea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GCA_000143785.1,ATCC 35910,"Level 6, Finished33.84x;7",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Chryseobacterium gleum,"HMP body site, Vaginal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATCC:35910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reference Genome,5562000,NaN,NaN,NaN,Homo sapiens-associated habitat ENVO:00009003,Chryseobacterium gleum F93,Newbler v. 2.0.0-Eclipse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homo sapiens,NaN,NaN,terrestrial biome ENVO:00000446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIGS.ba,NaN,biological product ENVO:02000043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://hmpdacc.org/doc/CommonGeneAnnotation_SO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATCC 35910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454-GS-FLX,NaN


In [51]:
merged = pd.merge(df, code_sra, how="right", left_on="assembly_accession", right_on="assembly_accession", indicator="merge")
merged["merge"].value_counts()

both          536
right_only      0
left_only       0
Name: merge, dtype: int64

In [52]:
merged.drop(columns=["merge", "sample_attribute"], inplace=True)

null_cols = merged.isnull().all() #REMOVE empty columns
merged = merged[merged.columns[~null_cols]]

merged.head()

,assembly_accession,strain,finishing strategy (depth of coverage),biotic_relationship,geo_loc_name,host,collected_by,host_disease,isolation_source,sample_name_x,isolation-source,sample_type,FDA CVM ID,culture_collection,assembly_method,collection_date,env_feature,synonym,Strain,SUBJECT_ID,Alias,type-material,misc_param,collection date,Isolation Site,Isolation source,country,note,GOLD Stamp ID,investigation_type,culture-collection,lat_lon,alt_elev,locus_tag_prefix,nat-host,assembly_method_version,env_material,model,sample_description,Sample Description,ArrayExpress-STRAIN_OR_LINE,ENA checklist,project_type,estimated_size,Temperature Optimum,isol_growth_condt,Motility,feature,project_name,assembly,num_replicons,isolate,depth,env_biome,altitude,ref_biomaterial,package,geographic location,ArrayExpress-StrainOrLine,ArrayExpress-Phenotype,ArrayExpress-SPECIES,INSDC center alias,geo-loc-name,environment,host_sex,elev,PublicAccession,completeness_estimated,geographic location (country and/or sea),latitude and longitude,specific_host,ArrayExpress-Species,ENA-FIRST-PUBLIC,INSDC center name,specific host,Body Sample Site,pH,biome,biomaterial_provider,isolate_name_alias,contamination_estimated,host scientific name,anonymized_name,ENA-LAST-UPDATE,INSDC first public,Cell Shape,BioSampleModel,environmental_sample,material,identified_by,host health state,INSDC last update,Diseases,strain_name_alias,genome_coverage,INSDC status,host taxid,isolation source,Gram Staining,Temperature Range,...,Historical Monobactam Produced,identification method,supplier_name,SRA accession,host_disease_outcome,Unique Monobactam Clusters Identified,ProjectAccession,Sample Name,Type Strain,Phenotypes,Gene Calling Method,temp,Title,investigation type,Host,rel_to_oxygen,host_age,Species,mapping_method,serovar,sop,Isolation Comments,finishing_strategy,Genus,mapping_method_version,environment (biome),attribute_package,metagenome_source,environment (feature),trophic_level,metagenomic,environment (material),Funding Program,quality_assessment_method,quality_assessment_method_version,geographic location (latitude),source_mat_id,host_tissue_sampled,geographic location (longitude),Host Name,subsrc_note,Host Health,value,plant-associated environmental package,project name,sequencing method,Body Sample SubSite,run_accession,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,experiment_ID,experiment_alias,experiment_accession,experiment_title,study_name,sample_name_y,design_description,library_name,library_strategy,library_source,library_selection,library_layout,library_construction_protocol,read_spec,platform,instrument_model,platform_parameters,experiment_attribute,sample_ID,sample_alias,sample_accession,taxon_id,common_name,description,sample_url_link,study_ID,study_alias,study_accession,study_title,study_type,study_abstract,center_project_name,study_description,study_url_link,study_attribute,submission_ID,submission_accession,submission_comment,submission_center,submission_lab,submission_date,sradb_updated
0,GCA_000023285.1,DSM 7271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type strain of Capnocytophaga ochracea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SRR013476,0,1093152,FWCBa,NaN,2019-11-28,9047.0,2866262.0,JGI,FWCBa,flow_count: 110 || flow_sequence: TACG || TCAG...,1051231,FWCBa,SRX002589,NaN,FWCB,FWCB,none provided,FWCB,WGS,GENOMIC,RANDOM,SINGLE -,NaN,READ_INDEX: 0; READ_CLASS: Technical Read; REA...,LS454,454 GS FLX,INSTRUMENT_MODEL: 

In [53]:
df.to_csv("SRA_metadata.csv", index=False)